# Training VAE for RGB Images

### Imports

In [ ]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedDataset

from models import FC_VAE
from train_vae import train, evaluate


### SETUP

In [ ]:
BATCH_SIZE = 32
EPOCHS = 50
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 10
GAMMA = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

features_file = "saved_features/saved_feat_I3D_25_dense_D1"

### TRAINING

In [ ]:
train_dataset = FeaturesExtendedDataset(features_file,'train')
train_loader_rgb = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)


model = FC_VAE(dim_input=1024, nz=64)
model.to(DEVICE)
print(f'Initial model device: {model.device}')

# Create Optimizer & Scheduler objects
optimizer = Adam(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train(model, optimizer, EPOCHS, DEVICE, train_loader_rgb, train_loader_rgb, BATCH_SIZE, scheduler)

torch.save(model.state_dict(), f'./saved_models/VAE_RGB/final_VAE_RGB_epoch_{EPOCHS}.pth')


### EVALUATION

In [ ]:
train_dataset = FeaturesExtendedDataset(features_file,'train')
test_dataset = FeaturesExtendedDataset(features_file,'test')
train_loader_rgb = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_loader_rgb = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

model = FC_VAE(dim_input=1024, nz=64)
model.to(DEVICE)
model.load_state_dict(torch.load(f'./saved_models/VAE_RGB/final_VAE_RGB_epoch_50.pth'))

reconstructed, originals = evaluate(model, DEVICE, train_loader_rgb)
reconstructed2, originals2 = evaluate(model, DEVICE, test_loader_rgb)

# Training VAE for EMG Signals

In [42]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedEMGDataset

from models import FC_VAE
from train_vae import train_emg, evaluate_emg
import pandas as pd
import numpy as np

### SETUP

In [50]:
BATCH_SIZE = 32
EPOCHS = 50
LR = 0.003
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 10
GAMMA = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

LSTM_features_file_train = "saved_features/NEW_EMG_Emb_LSTM_25_dense_D1_train.pkl"
LSTM_features_file_test = "saved_features/NEW_EMG_Emb_LSTM_25_dense_D1_test.pkl"
STAT_features_file_train = "saved_features/EMG_Emb_Stat_25_dense_D1_train.pkl"
STAT_features_file_test = "saved_features/EMG_Emb_Stat_25_dense_D1_test.pkl"

------ USING APPLE SILICON GPU ------


### TRAINING

In [44]:
train_dataset = FeaturesExtendedEMGDataset(LSTM_features_file_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True, multiprocessing_context='fork' if torch.backends.mps.is_available() else None)

for i in train_loader:
    print(i["features"].shape)
    break

torch.Size([32, 256])


In [ ]:
for (rgb_batch_idx, x) in enumerate(train_loader):
    print(x)
    break

In [51]:
model = FC_VAE(dim_input=256, nz=64)
model.to(DEVICE)
print(f'Initial model device: {model.device}')

# Create Optimizer & Scheduler objects
optimizer = Adam(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train_emg(model, optimizer, EPOCHS, DEVICE, train_loader, train_loader, BATCH_SIZE, scheduler)

torch.save(model.state_dict(), f'./saved_models/VAE_EMG_LSTM_NEW/NEW_final_VAE_EMG_STAT_epoch_{EPOCHS}.pth')

Initial model device: mps


280it [00:04, 57.93it/s]
2024-06-11 21:59:54 LOG INFO 	Epoch, 1, 	Average Loss: , 0.4263950584716694
280it [00:04, 58.70it/s]
2024-06-11 21:59:59 LOG INFO 	Epoch, 2, 	Average Loss: , 0.4987638844410792
280it [00:04, 59.40it/s]
2024-06-11 22:00:03 LOG INFO 	Epoch, 3, 	Average Loss: , 0.35013523910154576
280it [00:04, 58.75it/s]
2024-06-11 22:00:08 LOG INFO 	Epoch, 4, 	Average Loss: , 0.3265098321475222
280it [00:04, 58.71it/s]
2024-06-11 22:00:13 LOG INFO 	Epoch, 5, 	Average Loss: , 0.32591598746650535
280it [00:04, 58.59it/s]
2024-06-11 22:00:18 LOG INFO 	Epoch, 6, 	Average Loss: , 0.32472302911529405
280it [00:04, 59.22it/s]
2024-06-11 22:00:23 LOG INFO 	Epoch, 7, 	Average Loss: , 0.3250541244783709
280it [00:04, 58.66it/s]
2024-06-11 22:00:28 LOG INFO 	Epoch, 8, 	Average Loss: , 0.32557127258897256
280it [00:04, 57.42it/s]
2024-06-11 22:00:33 LOG INFO 	Epoch, 9, 	Average Loss: , 0.32512725400988773
280it [00:04, 59.68it/s]
2024-06-11 22:00:37 LOG INFO 	Epoch, 10, 	Average Loss: , 0.3

### EVALUATION

In [53]:
train_dataset = FeaturesExtendedEMGDataset(LSTM_features_file_train)
test_dataset = FeaturesExtendedEMGDataset(LSTM_features_file_test)
train_loader_emg = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_loader_emg = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

model = FC_VAE(dim_input=256, nz=64)
model.to(DEVICE)
model.load_state_dict(torch.load(f'./saved_models/VAE_EMG_LSTM_NEW/NEW_final_VAE_EMG_STAT_epoch_50.pth'))

reconstructed, originals = evaluate_emg(model, DEVICE, train_loader_emg, train_loader_emg)
reconstructed2, originals2 = evaluate_emg(model, DEVICE, test_loader_emg, test_loader_emg)

280it [00:02, 98.13it/s] 


Test Loss: 0.0101


31it [00:01, 22.54it/s]


Test Loss: 0.0088


# Fine Tuning Training [ RGB --> EMG ]

### IMPORTS

In [72]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedDataset, FeaturesExtendedEMGDataset, ActionNetEmgRgbDataset
from models import I3D
from models import EMG_Feature_Extractor
from utils.args import args
from omegaconf import OmegaConf
import tqdm
import pickle


from models import FC_VAE, LSTM_Emb_Classifier, EMG_Feature_Extractor
from train_vae import train_tuning, evaluate_tuning

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

------ USING APPLE SILICON GPU ------


In [75]:
# ---------------------- I3D ----------------------
# CONFIGURATION FOR I3D
conf_args = OmegaConf.load('configs/I3D_save_feat.yaml')
args = OmegaConf.merge(args, conf_args)

model_rgb = I3D(20, "RGB", args.models['RGB'], **args.models['RGB'].kwargs)
train_augmentations, test_augmentations = model_rgb.get_augmentation('RGB')
model_rgb.to("cpu")

# ---------------------- LSTM ----------------------
# Parametri del modello
input_dim = 16
hidden_dim = 300
embedding_dim = 256
output_dim = 20  # Definisci il numero di classi

model_emg = LSTM_Emb_Classifier(input_dim=input_dim, hidden_dim=hidden_dim, embedding_dim=embedding_dim, num_class=output_dim)
model_emg.load_state_dict(torch.load(f'./saved_models/LSTM_Emb_Classifier/NEW_final_LSTM_Emb_25_epoch_10.pth'))
model_emg.to(DEVICE)

# ---------------------- DATASET ----------------------
# DATASET 25 FRAMES PER CLIP AND 1 SAMPLE PER BATCH
num_frames = 25
num_clips = 1
batch_size = 1
dataset = ActionNetEmgRgbDataset('train', num_frames, num_clips, True, './action-net', 'action-net/saved_emg', "action-net/saved_RGB", 2, train_augmentations)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=False)
len(data_loader)



2024-06-11 22:26:28 LOG INFO Loading Kinetics weights I3D
2024-06-11 22:26:28 LOG INFO  * Skipping Logits weight for 'logits.conv3d.weight'
2024-06-11 22:26:28 LOG INFO  * Skipping Logits weight for 'logits.conv3d.bias'
/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


186

### FEATURE EXTRACTION ACTION NET s04

In [76]:
embeddings_rgb = []
embeddings_emg = []


model_rgb.train(False)
with torch.no_grad():
    for (idx, (emg,rgb,l)) in enumerate(data_loader):
        e = emg.reshape(batch_size, num_clips, num_frames, -1)  #(num_batch, num_clips, num_frames, num_features)
        # print(e)                  # torch.Size([1, 1, 25, 16])
        emg_input = e[0].float() # torch.Size([25, 16])
        emg_input = emg_input.to(DEVICE)

        batch, _, height, width = rgb.shape
        rgb_reshape = rgb.reshape(1, num_clips, num_frames, -1, height, width)
        rgb_permute = rgb_reshape.permute(1, 0, 3, 2, 4, 5)
        rgb_input = rgb_permute[0].to('cpu')    # CLIP
        # print(rgb_permute.shape)   torch.Size([1, 1, 3 (RGB), 25, 224, 224])
        # print(l)                   tensor([16])
        # ---------------------- RGB EXTRACTION ----------------------
        output_rgb, feat_rgb = model_rgb(rgb_input)
        feat_rgb = feat_rgb["features"]
        sample_rgb = feat_rgb[0]      # torch.Size([1, 1024])
        embeddings_rgb.append(sample_rgb)

        # ---------------------- EMG EXTRACTION ----------------------
        outputs_emg, feat_emg = model_emg(emg_input)
        sample_emg = feat_emg[0]      # torch.Size([64])
        embeddings_emg.append(sample_emg)

        # ---------------------- EMG STAT EXTRACTION ----------------------
        #embeddings = EMG_Feature_Extractor(emg_input[0])
        #embeddings_emg.append(embeddings)

print(len(embeddings_emg))
print(len(embeddings_emg))

features_emg = "saved_features/NEW_FINE_TUNING_emg_LSTM_s04_TRAIN.pkl"
features_rgb = "saved_features/NEW_FINE_TUNING_rgb_s04_TRAIN.pkl"

with open(features_rgb, 'wb') as f:
    pickle.dump(embeddings_rgb, f)

with open(features_emg, 'wb') as f:
    pickle.dump(embeddings_emg, f)


/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <0EB69795-4559-3C98-9EA1-35B6A988BB99> /Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <E4E2FFCA-031E-3974-A7B0-45408D7F4956> /Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")
/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image 

186
186


### SETUP

In [103]:
BATCH_SIZE = 32
EPOCHS = 50
LR = 0.0001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 50
GAMMA = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")


------ USING APPLE SILICON GPU ------


In [80]:
import pandas as pd
from utils.loaders import FeaturesTuningDataset

rgb = FeaturesTuningDataset("./saved_features/NEW_FINE_TUNING_rgb_s04_TRAIN.pkl")
emg = FeaturesTuningDataset("./saved_features/NEW_FINE_TUNING_emg_LSTM_s04_TRAIN.pkl")
rgb_test = FeaturesTuningDataset("./saved_features/NEW_FINE_TUNING_rgb_s04_TEST.pkl")
emg_test = FeaturesTuningDataset("./saved_features/NEW_FINE_TUNING_emg_LSTM_s04_TEST.pkl")
print(f'RGB dataset size: {len(rgb_test)}')

loader_rgb = DataLoader(rgb, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False, pin_memory=False)
loader_emg = DataLoader(emg, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False, pin_memory=False)
loader_rgb_test = DataLoader(rgb_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False, pin_memory=False)
loader_emg_test = DataLoader(emg_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False, pin_memory=False)

# print device loaders
for (rgb_batch_idx, x) in enumerate(loader_rgb):
    print(x.device)  # result --> cpu
    break

RGB dataset size: 19
cpu


### TRAINING

In [104]:
from train_vae import loss_function
from utils.logger import setup_logger
from torch.autograd import Variable
from torch.optim import Adam
from train_vae import train_tuning, evaluate_tuning

# LSTM EMG dimension is 64 
# STAT EMG dimension is 112
model_finetune = FC_VAE(dim_input=1024, nz=64, dim_output=256)
model_finetune.to(DEVICE)

# Carica i pesi del modello RGB per l'encoder
checkpoint_rgb = torch.load('./saved_models/VAE_RGB/final_VAE_RGB_epoch_50.pth', map_location=DEVICE)
# Rimuovi il prefisso 'encoder.' dalle chiavi dello state_dict
checkpoint_rgb = {k.replace('encoder.', ''): v for k, v in checkpoint_rgb.items() if 'encoder' in k}
model_finetune.encoder.load_state_dict(checkpoint_rgb)

# Carica i pesi del modello EMG per il decoder
checkpoint_emg = torch.load('./saved_models/VAE_EMG_LSTM_NEW/NEW_final_VAE_EMG_STAT_epoch_50.pth', map_location=DEVICE)
checkpoint_emg = {k.replace('decoder.', ''): v for k, v in checkpoint_emg.items() if 'decoder' in k}
model_finetune.decoder.load_state_dict(checkpoint_emg)

# Create Optimizer & Scheduler objects
optimizer = Adam(model_finetune.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train_tuning(model_finetune, optimizer, EPOCHS, DEVICE, loader_rgb, loader_emg, BATCH_SIZE, scheduler)
evaluate_tuning(model_finetune, DEVICE, loader_rgb_test, loader_emg_test)

torch.save(model_finetune.state_dict(), f'./saved_models/VAE_Fine_Tuninng/VAE_RGB_to_EMG_epoch_{EPOCHS}.pth')


2024-06-11 22:37:18 LOG INFO 	Epoch, 1, 	Average Loss: , 0.8024037688970566
2024-06-11 22:37:18 LOG INFO 	Epoch, 2, 	Average Loss: , 0.7427804488688707
2024-06-11 22:37:19 LOG INFO 	Epoch, 3, 	Average Loss: , 0.7411344680935145
2024-06-11 22:37:19 LOG INFO 	Epoch, 4, 	Average Loss: , 0.7384763456881046
2024-06-11 22:37:19 LOG INFO 	Epoch, 5, 	Average Loss: , 0.736753336340189
2024-06-11 22:37:19 LOG INFO 	Epoch, 6, 	Average Loss: , 0.7353636845946312
2024-06-11 22:37:19 LOG INFO 	Epoch, 7, 	Average Loss: , 0.73434037566185
2024-06-11 22:37:19 LOG INFO 	Epoch, 8, 	Average Loss: , 0.7338790915906429
2024-06-11 22:37:19 LOG INFO 	Epoch, 9, 	Average Loss: , 0.7334365762770176
2024-06-11 22:37:19 LOG INFO 	Epoch, 10, 	Average Loss: , 0.7331637397408486
2024-06-11 22:37:19 LOG INFO 	Epoch, 11, 	Average Loss: , 0.7331827685236931
2024-06-11 22:37:19 LOG INFO 	Epoch, 12, 	Average Loss: , 0.7328770682215691
2024-06-11 22:37:20 LOG INFO 	Epoch, 13, 	Average Loss: , 0.732866282761097
2024-06-11 2

Test Loss: 0.0036
